In [ ]:
# !pip install dill
# !pip install lime

In [ ]:
import pandas as pd
import numpy as np
# import joblib
# import dill
# from lime.lime_tabular import LimeTabularExplainer
from google.colab import userdata
%run "{userdata.get('gral_funcs_box_data')}"
%run "{userdata.get('funcs_model_2_short')}"
%run "{userdata.get('cleaner_raw_data')}"
%run "{userdata.get('clases_predictor')}"
%xmode #verbose
import warnings
import pdb
# pdb.set_trace()
warnings.filterwarnings('ignore')
cluster = joblib.load(userdata.get('cluster'))
scaler = joblib.load(userdata.get('scaler'))
model = joblib.load(userdata.get('model'))
with open(userdata.get('explainer'), 'rb') as file:
  explainer = dill.load(file)
pd.set_option('display.max_columns', 50)
# lectura de la data nueva
bo = pd.read_csv(userdata.get('boxers_info'))
fi = pd.read_csv(userdata.get('fight_info_real'))
fr = pd.read_csv(userdata.get('fight_results_real'))
mp = pd.read_csv(userdata.get('mades_predictions'))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 987.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=d63a721d331fe3a1c8dbfe315fc3ba00ba376bf869c02d8d03189edcaf5e2acb
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/basic.py:375: UserWarning: Error changing user exception modes.
Unrecognized mode in FormattedTB: <#verbose>
Valid modes: ['Plain', 'Context', 'Verbose', 'Minimal']
  warn('Error changing %s exception modes.\n%s' %


In [ ]:
# merge and split new data
ndf, new_id_df, new_other_cols = box_data_merger_spliter(bo,fi,fr)

# separate ndf to get the columns with res nan values
data = ndf[ndf.result.isna()]

In [ ]:
def data_without_prediction(df_predictions, df_to_predict):
    temp = pd.merge(df_to_predict, df_predictions[['fight_id']], on='fight_id', how='left', indicator=True)
    temp = temp[temp['_merge'] == 'left_only'].drop('_merge', axis=1)
    return temp

# Assuming `ndf` is defined elsewhere
df_preds = pd.read_csv(userdata.get('mades_predictions'))
data = data_without_prediction(df_preds, ndf)

In [ ]:
data

,fight_id,result,endround,endtype,boxer1,boxer2,birth_b1,height_b1,b1_w,b1_wk,b1_d,b1_l,b1_lk,b1_bet,len_text_boxer1,birth_b2,height_b2,b2_w,b2_wk,b2_d,b2_l,b2_lk,b2_bet,len_text_boxer2
5,5.0,cancelada,NaN,NaN,robson conceicao,humberto galindo,1996,170,17.0,8.0,0.0,2.0,0.0,1.03,7632,1999,173,14.0,11.0,1.0,3.0,1.0,1.29,0
16,17.0,draw,12.0,draw,jessica mccaskill,sandy ryan,1984,168,12.0,5.0,0.0,3.0,0.0,1.10,0,1993,177,6.0,2.0,0.0,1.0,0.0,1.11,980
27,28.0,draw,NaN,draw,terri harper,cecilia braekhus,1997,174,14.0,6.0,1.0,1.0,1.0,1.06,6569,1981,171,37.0,9.0,0.0,2.0,0.0,1.22,3664
32,33.0,cancelada,NaN,NaN,joshua buatsi,dan azeez,1993,188,17.0,12.0,0.0,0.0,0.0,1.06,3957,1989,178,20.0,12.0,0.0,0.0,0.0,1.16,2269
45,48.0,cancel,NaN,NaN,anthony joshua,dillian whyte,1989,198,24.0,22.0,0.0,3.0,1.0,1.04,54364,1988,193,29.0,19.0,0.0,3.0,3.0,1.17,36383
52,55.0,draw,NaN,NaN,euri cedeno,dayan despestre,2000,183,6.0,6.0,0.0,0.0,0.0,1.02,0,1999,188,1.0,1.0,0.0,0.0,0.0,13.00,0
80,84.0,d,NaN,NaN,antonio mireles,skylar lacey,1997,206,8.0,7.0,0.0,0.0,0.0,1.25,0,1994,201,7.0,5.0,0.0,0.0,0.0,4.00,0


### making predictions

In [ ]:
predictor = Model_predictions(cluster, scaler, model, explainer)
preds = predictor.make_prediction(data, new_id_df)

> <ipython-input-183-680f0a4ad0ee>(29)confidence_check()
     27   def confidence_check(self, data):
     28     pdb.set_trace()
---> 29     df = data.copy()
     30     mes1_list = []
     31     mes2_list = []

ipdb> c


In [ ]:
preds.shape

(11, 26)

### logging data

In [ ]:
logger = Predictions_logger(userdata.get('mades_predictions'))

In [ ]:
logger.record_data(preds)

{'cluster': 3, 'boxer1_pred': 1, 'prob_win': 0.944, 'data_cuant': 'fail 10 % of 285 (warning: few data)(val, test)', 'confidence': ''}
{'boxer1': 'jermall charlo', 'boxer2': 'jose benavidez', 'dif_bet': '1.12 - 6.0 = -4.88'}
{'dif_len_text': '15408 - 2033 = 13375', 'dif_edad': '1990 - 1992 = -2', 'dif_height': '185 - 188 = -3'}
{'dif_w': '32.0 - 28.0 = 4.0', 'dif_wk_perc': '0.69 - 0.68 = 0.01', 'dif_l': '0.0 - 2.0 = -2.0', 'dif_lk_perc': '0.0 - 1.0 = -1.0'}
{'modelo': 'SVC(probability=True)', 'explanation': {'birth_b1 > 0.79': 0.2068, 'len_text_boxer1 > -0.02': 0.123, 'height_b2 > 0.29': -0.1038, '-0.38 < len_text_boxer2 <= -0.14': 0.1019, 'cluster > 0.12': 0.0907, '-0.81 < b1_lk <= -0.38': 0.0636, 'b1_mas_win_perc > 1.09': -0.0347, 'b1_mas_peleas > 0.84': -0.0347, 'b1_menos_loss_perc > 1.14': -0.0347, 'b2_invicto > 0.50': -0.0347}}

Opinion personal (1 - boxer1 win, 0 - boxer1 loss, c - close): 1
Saved data

{'cluster': 0, 'boxer1_pred': 0, 'prob_win': 0.484, 'data_cuant': 'fail 19 % 

In [ ]:
preds2 = logger.read_file()

In [ ]:
preds2.shape

(115, 23)

In [ ]:
# logger.delete_row(64)

Proced to delete record
{'fight_id': 64.0, 'self_pred': 0, 'cluster': 0, 'boxer1_pred': 1, 'prob_win': 0.935, 'data_cuant': 'fail 19 % of 1150 (val, test)', 'confidence': nan, 'boxer1': 'yoanki urrutia', 'boxer2': 'fiodor czerkaszyn', 'dif_bet': '3.1 - 1.36 = 1.74', 'dif_len_text': '0 - 0 = 0', 'dif_edad': '1996 - 1990 = 6', 'dif_height': '188 - 170 = 18', 'dif_w': '13.0 - 22.0 = -9.0', 'dif_wk_perc': '0.38 - 0.64 = -0.25', 'dif_l': '0.0 - 1.0 = -1.0', 'dif_lk_perc': '0.0 - 0.0 = 0.0', 'modelo': 'SVC(probability=True)', 'b1_id': 6553, 'b2_id': 6363, 'explanation': "{'birth_b1 > 0.79': 0.1979, 'b1_wk <= -0.66': -0.1433, 'cluster <= 0.12': -0.0968, '-0.42 < len_text_boxer2 <= -0.38': 0.089, 'b1_more_fame <= -1.62': -0.0611, 'height_b1 > 0.22': 0.056, 'b1_momios_menores <= -1.02': -0.0559, '-0.81 < b1_lk <= -0.38': 0.0549, 'b2_invicto > 0.50': 0.054, 'len_text_boxer1 <= -0.48': -0.0534}"}
 (y/n): y
Saved data
Row deleted successfully



### export predictions

In [12]:
from datetime import datetime, timedelta
import pytz
import pdb
    # pdb.set_trace()

class CloseException(Exception):
  pass

class Box_reports:
  '''
  Class to get the predictions and results of predictions
  formated to share, and to log results of fights.
  '''
  def __init__(self, mades_predictions_file, fights_info_file, fights_results_file):
    '''
    Constructor method.
    Parameters:
      mades_predictions_file: File with the predictions mades.
      fights_info_file: File with with the info of the fights.
      fights_results_file: File with the results of the fights.
    '''
    self.mp_file = mades_predictions_file
    self.fi_file = fights_info_file
    self.fr_file = fights_results_file

  def authorization(self, old_data, intend, new_data):
    autho = input(f'Proceed with {intend}? (y/n):')
    if autho == 'y':
      new_data.to_csv(old_data, index=False)
      return 1
    else:
      return 0

  def closer(self, message):
    res = input(f'{message}')
    if res == 'c':
      raise CloseException()
    else:
      return res

  def read_file(self):
    mp = pd.read_csv(self.mp_file)
    fi = pd.read_csv(self.fi_file)
    fr = pd.read_csv(self.fr_file)
    temp = mp.merge(fi[['fight_id','date_fight','date_extract','b1_bet','b2_bet']], how='left', on='fight_id')
    temp = temp.merge(fr, how='left', on='fight_id')
    return temp

  def get_fights_already_happen(self, data):
    tz = pytz.timezone('Etc/GMT+6') # Etc/GMT+6 corresponde a GMT-6
    now = datetime.now(tz) # Obtener la fecha y hora actual en GMT-6
    now_str = now.strftime("%Y-%m-%d")
    temp = data[data.date_extract.notna()]
    temp = temp[temp.date_fight < now_str]
    return temp

  def get_fights_not_happen_yet(self, data):
    tz = pytz.timezone('Etc/GMT+6') # Etc/GMT+6 corresponde a GMT-6
    now = datetime.now(tz) # Obtener la fecha y hora actual en GMT-6
    now_str = now.strftime("%Y-%m-%d")
    temp = data[data.date_extract.notna()]
    temp = temp[temp.date_fight >= now_str]
    return temp

  def get_value(self, df_row_col):
    val = df_row_col.values[0]
    return val

  def logger_res(self, fight_id, result, endtype, endround):
    temp = pd.read_csv(self.fr_file)
    temp.loc[fi.fight_id == fight_id, 'result'] = result
    temp.loc[fi.fight_id == fight_id, 'endtype'] = endtype
    temp.loc[fi.fight_id == fight_id, 'endround'] = endround
    res = self.authorization(self.fr_file, 'add results', temp)
    return res

  def no_result_data(self, data):
    temp = data.copy()
    temp = temp[temp.result.isna()]
    return temp

  def log_results_fights_already_happen(self):
    '''
    Method to log the result of the fights that already happen
    and have no result yet.
    '''
    temp = self.read_file()
    temp = self.get_fights_already_happen(temp)
    temp = self.no_result_data(temp)
    if temp.shape[0] == 0:
      print("All the fights have result recorded.")
    else:
      for id in temp.fight_id:
        tempo = temp[temp.fight_id == id, :]
        print(f'Add records (close=c):\nid fight: {self.get_value(tempo.fight_id)}, self pred: {self.get_value(tempo.self_pred)},\
        model pred: {self.get_value(tempo.boxer1_pred)}, proba: {self.get_value(tempo.prob_win)},\n\
        boxer 1: {self.get_value(tempo.boxer1)}, boxer2: {self.get_value(tempo.boxer2)}, date fight: {self.get_value(tempo.date_fight)}\n')
        result = self.closer('Result (w/l): ')
        endtype = self.closer('Endtype (k,d): ')
        endround = self.closer('Endround: ')
        res = self.logger_res(id, result, endtype, endround)
        if res == 1:
          print('Saved data.\n')
        else:
          print('Denied.\n')

  def renames_replaces_to_show(self, data):
    temp = data.copy()
    for i in ['self_pred', 'boxer1_pred']:
      temp[i].replace({'1': 'gana boxeador 1', '0': 'pierde boxeador 1',
                        1: 'gana boxeador 1', 0: 'pierde boxeador 1'}, inplace=True)
    temp.rename(columns={'self_pred':'predicción_procesada',
                        'boxer1_pred':'predicción_del_modelo',
                        'prob_win':'probabilidad',
                        'data_cuant':'cantidad_datos_por_cluster',
                        'confindence':'confianza_de_predicción'}, inplace=True)
    return temp

  def date_begin_date_end(self, data, begin, end):
    temp = data[(data.date_fight >= begin) & (data.date_fight < end)]
    return temp

  def show_preds(self, begin='2012-1-1', plus_days=30):
    '''
    Method to export predictions in a csv format.
    Parameters:
      begin (def - 2012-1-1): str. Begin date of the data to show.
      plus_days (def - 30): int. Days to add to the current date to show information at the future date.
    '''
    current_date = datetime.now().date()
    current_date = datetime.now().date()
    future_date = current_date + timedelta(days=plus_days)
    temp = self.read_file()
    temp = self.get_fights_not_happen_yet(temp)
    temp = self.date_begin_date_end(temp, begin, str(future_date))
    temp = temp[['fight_id','boxer1','boxer2','self_pred','date_fight','prob_win','boxer1_pred','b1_bet','b2_bet','data_cuant','confidence']]
    temp = self.renames_replaces_to_show(temp).sort_values('date_fight')
    temp.to_csv(f'preds_{current_date}.csv', index=False)

  def show_results(self, begin='2012-1-1', end=0):
    '''
    Method to export results of predisctions in csv format.
    Parameters:
      begin (def - 2012-1-1): str. Begin date of the data to show.
      end (def - current date): str. End date of the data to show.
    Returns:
      temp: Dataframe with the processed info.
    '''
    current_date = datetime.now().date()
    if end == 0:
      end = str(current_date)
    temp = self.read_file()
    temp = self.get_fights_already_happen(temp)
    temp = self.date_begin_date_end(temp, begin, end)
    temp = temp[['fight_id','boxer1','boxer2','result','self_pred','date_fight','boxer1_pred','prob_win','b1_bet','b2_bet','data_cuant','confidence']]
    temp = self.renames_replaces_to_show(temp)
    temp.result.replace({'1':'gano boxeador 1','0':'perdio boxeador 1','w':'gano boxeador 1','l':'perdio boxeador 1','d':'empate'},
                        inplace=True)
    temp = temp.rename(columns={'result':'resultado'}).sort_values('date_fight')
    temp.to_csv(f'preds_results_{current_date}.csv', index=False)
    return temp

  def analyse_results(self, begin='2012-1-1', end=0):
    '''
    Method to analyse the profit and results of the predictions.
    Parameters:
      begin (def - 2012-1-1): str. Begin date of the data to show.
      end (def - current date): str. End date of the data to show.
    Returns:
      temp: Dataframe with the processed info.
    '''
    temp = self.show_results(begin, end)
    for i in ['opinion_sobre_predicción', 'predicción_del_modelo']:
      temp[i].replace({'gana boxeador 1':1, 'pierde boxeador 1':0}, inplace=True)
    temp.result.replace({'gano boxeador 1':1, 'perdio boxeador 1':0, 'empate':-1}, inplace=True)
    temp['goodpred'] = temp.result == temp['opinion_sobre_predicción']
    temp['model_goodpred'] = temp.result == temp['predicción_del_modelo']
    lista = []
    for op, bet1, bet2 in zip(temp['opinion_sobre_predicción'], temp.b1_bet, temp.b2_bet):
      if op == 1:
        lista.append(bet1)
      elif op == 0:
        lista.append(bet2)
      else:
        print(f'Unacceptable value in opinion: {op}')
        return 0
    lista2 = []
    for gp, bet in zip(temp.goodpred, lista):
      if gp == True:
        lista2.append(bet - 1)
      elif gp == False:
        lista2.append(-1)
      else:
        print(f'Something happend: {gp}')
    temp['profit'] = lista2
    temp['acum_profit'] = temp.profit.cumsum()
    return temp

In [13]:
report = Box_reports(userdata.get('mades_predictions'),
                     userdata.get('fight_info_real'),
                     userdata.get('fight_results_real'))

In [ ]:
report.log_results_fights_already_happen()

All the fights have result recorded.


In [14]:
report.show_preds()

In [15]:
pd.read_csv('/content/preds_2023-11-22.csv')

,fight_id,boxer1,boxer2,predicción_procesada,date_fight,probabilidad,predicción_del_modelo,b1_bet,b2_bet,cantidad_datos_por_cluster,confidence
0,103.0,skye nicolson,lucy wildheart,gana boxeador 1,2023-11-25,0.905,gana boxeador 1,1.100,7.000,"fail 19 % of 1150 (val, test)",NaN
1,104.0,gary cully,reece mould,gana boxeador 1,2023-11-25,0.977,gana boxeador 1,1.200,4.500,"fail 19 % of 1150 (val, test)",NaN
2,105.0,paddy donovan,danny ball,gana boxeador 1,2023-11-25,0.947,gana boxeador 1,1.250,4.000,"fail 19 % of 1150 (val, test)",NaN
3,106.0,subriel matias,shohjahon ergashev,gana boxeador 1,2023-11-26,0.778,gana boxeador 1,1.250,4.000,"fail 19 % of 1150 (val, test)",NaN
4,115.0,jermall charlo,jose benavidez,gana boxeador 1,2023-11-26 00:00:00,0.944,gana boxeador 1,1.125,6.000,"fail 10 % of 285 (warning: few data)(val, test)",NaN
5,116.0,hector luis garcia,lamont roach,gana boxeador 1,2023-11-26 00:00:00,0.484,pierde boxeador 1,1.333,3.400,"fail 19 % of 1150 (val, test)","WARNING: .45 to .65, fail 50 %"
6,107.0,sean mccomb,sam maxwell,pierde boxeador 1,2023-12-02,0.939,gana boxeador 1,1.250,4.000,"fail 19 % of 1150 (val, test)",NaN
7,108.0,caoimhin agyarko,troy williamson,pierde boxeador 1,2023-12-02,0.698,gana boxeador 1,1.800,2.000,"fail 19 % of 1150 (val, test)",NaN
8,109.0,gavin gwynne,emiliano marsili,gana boxeador 1,2023-12-02,0.957,gana boxeador 1,1.125,6.000,"fail 19 % of 1150 (val, test)",NaN
9,110.0,tyrone mckenna,lewis crocker,gana boxeador 1,2023-12-02,0.649,gana boxeador 1,3.750,1.286,"fail 19 % of 1150 (val, test)","WARNING: .45 to .65, fail 50 %"


In [ ]:
temp = report.show_results()

In [ ]:
check_res(temp)#.acum_profit.plot()

,fight_id,boxer1,boxer2,result,opinion_sobre_predicción,date_fight,predicción_del_modelo,probabilidad,b1_bet,b2_bet,cantidad_datos_por_cluster,confidence,goodpred,model_goodpred,profit,acum_profit
71,80.0,souleymane cissokho,isaias lucero,1,1,2023-11-04,1,0.864,1.125,6.500,"fail 19 % of 1150 (val, test)",NaN,True,True,0.125,0.125
72,81.0,julissa alejandra guzman,ramla ali,0,0,2023-11-04,1,0.972,1.666,2.250,"fail 19 % of 1150 (val, test)",NaN,True,False,1.250,1.375
73,82.0,michal cieslak,tommy mccarthy,1,1,2023-11-04,1,0.988,1.286,3.500,"fail 19 % of 1150 (val, test)",NaN,True,True,0.286,1.661
74,85.0,omar rosario,angel rebollar,0,1,2023-11-04,1,0.929,1.050,9.000,"fail 19 % of 1150 (val, test)",NaN,False,False,-1.000,0.661
75,86.0,yokasta valle,anabel ortiz,1,1,2023-11-04,1,0.630,1.167,5.000,"fail 19 % of 1150 (val, test)","WARNING: .45 to .65, fail 50 %",True,True,0.167,0.828
76,87.0,javier martinez,isaiah wise,1,1,2023-11-04,1,0.874,1.125,6.000,"fail 19 % of 1150 (val, test)",NaN,True,True,0.125,0.953
77,88.0,jonathan guidry,jesus escalera,1,1,2023-11-04,1,0.904,1.200,4.500,"fail 19 % of 1150 (val, test)",NaN,True,True,0.200,1.153
78,89.0,lindolfo delgado,luis hernandez ramos,1,1,2023-11-04,1,0.719,1.167,5.000,"fail 19 % of 1150 (val, test)",NaN,True,True,0.167,1.320
79,90.0,henry lebron,william foster iii,1,0,2023-11-05,0,0.090,2.100,1.727,"fail 19 % of 1150 (val, test)",NaN,False,False,-1.000,0.320
80,91.0,trevor bryan,cassius chaney,0,0,2023-11-05,0,0.385,2.100,1.727,"fail 19 % of 1150 (val, test)","warning: .35 to .45, fail 30 %",True,True,0.727,1.047
